#### Test att hämta bilder och söka i SPA
* [Denna notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20Litteraturbanken.ipynb) 

Tanken att se vilka möjligheter som finns
* skapa ett script --> ett klick för att ladda upp bilder se test [spa2commons](https://commons.wikimedia.org/wiki/User:Salgo60/spa2commons.js) - GITHUB [salgo60/spa2Commons](https://github.com/salgo60/spa2Commons) 
* att ta en plats som en kyrkogård eller en koppling Litteraturbanken och se vad som kan matchas
* att stämma av länkar SPA -> WIkipedia att dessa finns i WIkidata med länk tillbaka via [P4819](https://www.wikidata.org/wiki/Property:P4819?uselang=sv)

## sv:Wikipedia
* kopplas till Litteraturbanken med en [mall](https://sv.wikipedia.org/wiki/Mall:Litteraturbanken) som visae användningen i Spårningskategorin [Litteraturbanken](https://sv.wikipedia.org/wiki/Kategori:Litteraturbanken)
  * [mallen är skriven](https://sv.wikipedia.org/w/index.php?title=Mall:Litteraturbanken&action=edit) så att identifieraren hämtas från [Wikidata egenskap P5101](https://www.wikidata.org/wiki/Property:P5101?uselang=sv) om inget annat anges

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-03-05 13:51:22.395091


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/latest.php"
url= "https://xn--portrttarkiv-kcb.se/endpoints/search.php"
url= "https://portrattarkiv.se/endpoints/search.php"  


<a href="SPA"></a>
## Icke kopplade personer till Svenskt Porträttarkiv P4819 som har  Litteraturbanken P5101

Testar att söka med lite olika parametrar för att få bättre precision i rankingen när jag söker mot Svenskt Porträttarkiv SPA
* personer hos Litteraturbanken [Property:P5101](https://www.wikidata.org/wiki/Property:P5101?uselang=sv)
  * bilderna i SPA är gamla --> utan copyright --> begränsa till sökning för födda innan 1920 
     * [SPARQL](https://w.wiki/6QWL)
        * 2023 03 05 ger 1290 träff


In [34]:
# query to get people with no picture in WIkidata but connected  Litteraturbanken
endpoint_url = "https://query.wikidata.org/sparql"
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# query people connected to Litteraturebank with no picture and born before 1920
# https://w.wiki/6QX5
queryLitt = """#title: Records in Wikidata connected to Litteraturbanken with no picture for people born before 1920
SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID) 
(CONCAT (?itemLabel," ",str(year(?birthDayWD))) AS ?search)
?birthDayWD
?firstnameLabel ?lastnameLabel
?item ?itemLabel 
(str(year(?birthDayWD)) AS ?BirthYearWD)
?Littauthorid
  WHERE {
  ?item wdt:P5101 ?Littauthorid. 
#  minus {?item wdt:P4819 ?c} 
  minus {?item wdt:P18 ?noimage }
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
    
    FILTER (?birthDayWD <= "1920-01-01T00:00:00Z"^^xsd:dateTime )
}"""

# query for all Littbank without a picture in WD but article in Wikidata
# https://w.wiki/6QWw
queryArticle = """SELECT distinct (CONCAT (?itemLabel," ",str(year(?birthDayWD))) AS ?search)
?birthDayWD
?firstnameLabel ?lastnameLabel
?item ?itemLabel ?wikipedia
(str(year(?birthDayWD)) AS ?BirthYearWD)
  WHERE {
  ?item wdt:P5101 ?Litt. 
  minus {?item wdt:P18 ?d }
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  {
      ?wikipedia schema:about ?item .
      ?wikipedia schema:inLanguage "sv" .
      ?wikipedia schema:isPartOf <https://sv.wikipedia.org/> .
    }  

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
} order by ?itemLabel """

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


#SPALittdf = get_sparql_dataframe(endpoint_url, query)
SPALittdf = get_sparql_dataframe(endpoint_url, queryLitt)
SPALittdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   WikidataID      1287 non-null   object
 1   search          1287 non-null   object
 2   birthDayWD      1287 non-null   object
 3   firstnameLabel  1016 non-null   object
 4   lastnameLabel   608 non-null    object
 5   item            1287 non-null   object
 6   itemLabel       1287 non-null   object
 7   BirthYearWD     1287 non-null   object
 8   Littauthorid    1287 non-null   object
dtypes: object(9)
memory usage: 90.6+ KB


In [35]:
SPALittdf

,WikidataID,search,birthDayWD,firstnameLabel,lastnameLabel,item,itemLabel,BirthYearWD,Littauthorid
0,Q5601467,Johannes Campanius 1601,1601-08-15T00:00:00Z,John,None,http://www.wikidata.org/entity/Q5601467,Johannes Campanius,1601,CampaniusJJ
1,Q5603107,Gottfrid Carlsson 1887,1887-12-18T00:00:00Z,Henrik,Carlsson,http://www.wikidata.org/entity/Q5603107,Gottfrid Carlsson,1887,CarlssonGottfrid
2,Q5603107,Gottfrid Carlsson 1887,1887-12-18T00:00:00Z,Oscar,Carlsson,http://www.wikidata.org/entity/Q5603107,Gottfrid Carlsson,1887,CarlssonGottfrid
3,Q5603107,Gottfrid Carlsson 1887,1887-12-18T00:00:00Z,Gottfrid,Carlsson,http://www.wikidata.org/entity/Q5603107,Gottfrid Carlsson,1887,CarlssonGottfrid
4,Q5604633,Samuel Niclas Casström 1763,1763-12-04T00:00:00Z,Samuel,None,http://www.wikidata.org/entity/Q5604633,Samuel Niclas Casström,1763,CasströmSN
...,...,...,...,...,...,...,...,...,...
1282,Q115957420,Alfred Roman 1829,1829-01-01T00:00:00Z,None,None,http://www.wikidata.org/entity/Q115957420,Alfred Roman,1829,RomanA
1283,Q115957550,Reinhold Götberg 1835,1835-04-30T00:00:00Z,None,None,http://www.wikidata.org/entity/Q115957550,Reinhold Götberg,1835,GötbergR
1284,Q115958630,Salomon Salomonsson 1867,1867-01-01T00:00:00Z,None,None,http://www.wikidata.org/entity/Q115958630,Salomon Salomonsson,1867,SalomonssonS
1285,Q115958676,Emma Eugenia Matilda Ahlberg 1833,1833-12-31T00:00:00Z,None,None,http://www.wikidata.org/entity/Q115958676,Emma Eugenia Matilda Ahlberg,1833,RogbergM


In [36]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [37]:
from wikidata.client import Client
from tqdm.notebook import tqdm 
import urllib3, json
http = urllib3.PoolManager() 
SPAdetail = "https://portrattarkiv.se/details/"
url= "https://portrattarkiv.se/endpoints/search.php"  
urlbasePic = "https://portrattarkiv.se/endpoints/file.php?id="

def getdfScore(panddf):
    dfScore = pd.DataFrame(columns=['all','WikidataID','spaid','Littauthorid','score',
                                    'FirstNameWD','LastNameWD','BirthDayWD',
                                    'FirstNameSPA','LastNameSPA','BirthDaySPA','searchStr'])
    #for index, row  in SPALittdf.iterrows():
    for index, row in tqdm(panddf.iterrows(),total=panddf.shape[0]):
    
        allString=row["search"]
        wd=row["WikidataID"]
        Littauthorid=row["Littauthorid"]
        
        print("\n",allString,wd)
        encoded_body = json.dumps({
            "limit": "5",
            "from": "0",
            "firstname" : row["firstnameLabel"], 
            "lastname" : row["lastnameLabel"], 
            "birthyear":row["BirthYearWD"],            
            "all":allString
            
        })
    
        r = http.request('POST', url,
                 headers={'Content-Type': 'application/json'},
                 body=encoded_body)

        if r.status != 200:
            print(r.status)
            continue
    
        data = json.loads(r.data.decode('utf-8'),) 
        urls = []

        for h in data["hits"]["hits"]:
            spaid = h["_id"]
            #print(h)
            source = h["_source"]
            try:
                urlPicture = urlbasePic + spaid
                urls.append(urlPicture)
                score = h["_score"]
                FirstNameSPA = source["FirstName"]
                LastNameSPA = source["LastName"]
                BirthYearSPA = source["BirthYear"]
                print("\t\t",score,FirstNameSPA, " ", LastNameSPA, " - ", BirthYearSPA,SPAdetail+spaid, "\t", )
                dfScore = dfScore.append({
                'all' : allString,
                'wd' : wd,
                'Littauthorid' : Littauthorid,
                'spaid' : spaid,
                'score' : score,
                'FirstNameWD' : row["firstnameLabel"],
                'LastNameWD' : row["lastnameLabel"],
                'BirthYearWD' : row["BirthYearWD"],
                'FirstNameSPA' : FirstNameSPA,
                'LastNameSPA' : LastNameSPA,
                'BirthYearSPA' : BirthYearSPA
                },ignore_index=True)
            except Exception as e:
                print(f"{type(e).__name__} at line {e.__traceback__.tb_lineno} of {__file__}: {e}")
    return dfScore

In [38]:
dfScoreSPALitt = getdfScore(SPALittdf)
dfScoreSPALitt.info()

  0%|          | 0/1287 [00:00<?, ?it/s]


 Johannes Campanius 1601 Q5601467

 Gottfrid Carlsson 1887 Q5603107
		 40.213776 Gottfrid   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6R9w 	
		 39.32846 Gottfrid   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6R-A 	
		 25.88007 E G   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6Pqg 	
		 25.743935 Erik   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6QqQ 	
		 25.580647 Erik   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6Q4g 	

 Gottfrid Carlsson 1887 Q5603107
		 40.80401 Gottfrid   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6R9w 	
		 39.945587 Gottfrid   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6R-A 	
		 37.107048 Oscar   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6X5w 	
		 33.565414 Oscar   Carlsson  -  1887 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA6X6Q 	
		 33.562786 Oscar 


 Johan Eric Fant 1815 Q5725157
		 29.121262 Johan   Fast  -  1815 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUCjQ 	

 Per Fischier 1852 Q5731816
		 26.088505 Per   Gussenius  -  1852 https://portrattarkiv.se/details/1ha4xkDj-uAAAAAAAAAmXg 	
		 25.886486 Per   Sjölund  -  1852 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAXwA 	
		 25.86299 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADfmQ 	
		 25.710876 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABK5Wg 	
		 25.640894 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADifw 	

 Per Fischier 1852 Q5731816
		 26.088505 Per   Gussenius  -  1852 https://portrattarkiv.se/details/1ha4xkDj-uAAAAAAAAAmXg 	
		 25.886486 Per   Sjölund  -  1852 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAXwA 	
		 25.86299 Per   Lilliecreutz  -  1852 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADfmQ 	
		 25.710876 Per   Lilliecreutz  -  1852 ht


 Johan Gustaf Liljegren 1791 Q5951696
		 63.164345 J G   Liljegren  -  1791 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAADc3A 	

 Carl Johan Lohman 1694 Q5962508

 Johan Fredrik af Lundblad 1791 Q5966119

 Johan Fredrik af Lundblad 1791 Q5966119

 Hans Jakob Lundborg 1825 Q5966187
		 64.99777 Hans Jakob   Lundborg  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4wDQ 	
		 56.870396 Hans   Lundborg  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4vSw 	
		 56.77995 Hans   Lundborg  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABHRDw 	
		 56.27118 Hans   Lundborg  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4vRA 	
		 56.11027 Hans   Lundborg  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4vUQ 	

 Emanuel Mallén 1786 Q5973362

 John Morén 1871 Q5993473
		 13.664967 Johan   Nowén  -  1871 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAABMbg 	

 Johan Fredrik Muncktell 1764 Q5996948
		 76.43912 Johan   Muncktell  -  176

		 45.777985 Nils   Stromberg  -  1646 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2WjQ 	

 Nils Stromberg 1646 Q1346525
		 50.23616 Nils   Stromberg  -  1646 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2WjQ 	

 Per Ulrik Kernell 1797 Q1388455

 Johan Vibe 1748 Q1442709

 Olof Torén 1718 Q1493160

 Pehr Fjellström 1697 Q1523366

 Gustav Konrad Sus 1823 Q1556454
		 27.470535 Gustav   Rubin  -  1823 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAp6Q 	
		 15.690037 Gustaf   Klemming  -  1823 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABM6CQ 	
		 15.646865 Gustaf   Klemming  -  1823 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAABYoQ 	
		 15.616961 Gustaf   Wasastjerna  -  1823 https://portrattarkiv.se/details/ciLMGSqrYjAAAAAAAAAblQ 	
		 15.481089 Gustaf   Rydell  -  1823 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB4qrA 	

 Ignatius Mouradgea d'Ohsson 1740 Q1657363

 Ignatius Mouradgea d'Ohsson 1740 Q1657363

 Peder Månsson 1462 Q1663076

 Kathrine Aurel

...

		 32.057842 Carl   Moqvist  -  1825 https://portrattarkiv.se/details/I_2wtd5VefAAAAAAAABQmw 	
		 31.108763 Carl   Moqvist  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUEew 	
		 30.771605 Carl   Moqvist  -  1825 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2Vng 	
		 26.305086 Carl   Spaak  -  1825 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAACwrw 	
		 26.28684 Carl   Backman  -  1825 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABhNg 	

 Christian Gustafsson Roman 1709 Q110212249
		 32.90836 Christian   Hjärne  -  1709 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAC0NQ 	
		 32.66755 Christian   Hjärne  -  1709 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABe3Ng 	
		 18.072092 Kristian   Hjärne  -  1709 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAADI-A 	

 Daniel L. Hodell 1678 Q110212250
		 18.617392 Johan   Strömfelt  -  1678 https://portrattarkiv.se/details/mLh5P0pzYFAAAAAAAAA2fg 	
		 17.980616 Johan   Strömfelt  -  1678 https://portratt

		 18.744198 Carl   Neuman  -  1821 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAAtPQ 	
		 13.295908 Zacharias   Nilsson  -  1821 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAAEFRg 	
		 13.229286 Bror   Mörcke  -  1821 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABL3Gg 	
		 13.213725 Richard   Carlén  -  1821 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABiUw 	
		 13.209431 Carl   Cantzler  -  1821 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVwLQ 	

 Åke-Hugo Hanson 1917 Q114105719
		 18.532911 Gunnar   Hanson  -  1917 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAACbLg 	
		 18.532911 Olof   Hanson  -  1917 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABheDg 	
		 18.497715 Olof   Hanson  -  1917 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAC5KQ 	
		 17.912209 Gunnar   Hanson  -  1917 https://portrattarkiv.se/details/j6S8rJvx9lAAAAAAAAAdeg 	
		 17.76772 John   Hanson  -  1917 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABWTfg 	

 Josef Jons

		 17.108044 Klas   Eriksson  -  1872 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAAxPQ 	
		 16.812662 Otto   Eriksson  -  1872 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAAzQA 	
		 16.610352 Ollas   Eriksson  -  1872 https://portrattarkiv.se/details/TR3yO_GV8eAAAAAAAAAqUw 	

 Anna Maria Schunck 1811 Q114310256
		 20.66385 Anna   Eriksson  -  1811 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABU3pQ 	
		 18.564882 Anna   Schenström-Reuterskiöld  -  1811 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAhAA 	
		 16.232702 Nils   Flemin  -  1811 https://portrattarkiv.se/details/lzSYNK3ifoMAAAAAAAbchQ 	
		 15.699028 Elisabeth   Poppius-Berzelius  -  1811 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABznvQ 	
		 15.507296 Martina   Lamberg  -  1811 https://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAAAIPg 	

 Wiktor Hässler 1838 Q114310654
		 11.776998 Charles   Koch  -  1838 https://portrattarkiv.se/details/0piq-fX0h5AAAAAAAAA_kA 	
		 11.738893 Johan   Malmén  -  1838 h

		 36.822453 Arvid   Isberg  -  1850 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABm3w 	
		 35.104485 Arvid   Isberg  -  1850 https://portrattarkiv.se/details/nGoaRo7IPMAAAAAAAAAGdg 	
		 34.64518 Arvid   Isberg  -  1850 https://portrattarkiv.se/details/nGoaRo7IPMAAAAAAAAAGdw 	
		 32.46599 Arvid   Isberg  -  1850 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZ4Ug 	
		 31.32848 Ragnar   Isberg  -  1850 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZ3fw 	

 Nils Peter Nilsson 1819 Q115956203
		 28.493364 Nils   Nilsson  -  1819 https://portrattarkiv.se/details/wbbNj8LLWMAAAAAAAABm4w 	
		 22.363834 Nils   Nilsson  -  1819 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUBKA 	
		 22.076519 Nils   Nilsson  -  1819 https://portrattarkiv.se/details/seuNpYu6DKAAAAAAAABWGg 	
		 21.31474 Nils   Nilsson  -  1819 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUBOg 	
		 20.268488 Peter   Andersson  -  1819 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAA5kRQ 	

 Fredrik Sun

		 30.079666 Salomonsson   Salomon  -  1867 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAADibQ 	
		 29.709518 Salomonsson   Salomon  -  1867 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABhzcg 	
		 24.236582 Salle   Salomonsson  -  1867 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAAAgUg 	
		 20.812597 Salomon   Salmonson  -  1867 https://portrattarkiv.se/details/4e1scVvZ-EAAAAAAAAAjaA 	
		 20.524279 Harald   Salomon  -  1867 https://portrattarkiv.se/details/YB0QHyfj0hAAAAAAAADOMA 	

 Emma Eugenia Matilda Ahlberg 1833 Q115958676
		 20.404686 Eugenia   Eneqvist-Biondini  -  1833 https://portrattarkiv.se/details/WQknIBFpuuAAAAAAAAN35w 	
		 19.630756 Matilda   Johnson  -  1833 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACZvyw 	
		 19.292498 Emma   Sager  -  1833 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABU3eg 	
		 18.539536 Otto   Wahlberg  -  1833 https://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABUrVg 	
		 17.777443 Johan   Ahlström  -  1833 https://portrattar

In [39]:
dfScoreSPALitt.sort_values('score', ascending=False)
dfScoreSPALitt.groupby("wd")

In [40]:
dfScoreSPALitt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   all           2762 non-null   object
 1   WikidataID    0 non-null      object
 2   spaid         2762 non-null   object
 3   Littauthorid  2762 non-null   object
 4   score         2762 non-null   object
 5   FirstNameWD   1490 non-null   object
 6   LastNameWD    388 non-null    object
 7   BirthDayWD    0 non-null      object
 8   FirstNameSPA  2762 non-null   object
 9   LastNameSPA   2762 non-null   object
 10  BirthDaySPA   0 non-null      object
 11  searchStr     0 non-null      object
 12  BirthYearSPA  2756 non-null   object
 13  BirthYearWD   2762 non-null   object
 14  wd            2762 non-null   object
dtypes: object(15)
memory usage: 323.8+ KB


In [41]:
dfScoreSPALitt.head(200)


,all,WikidataID,spaid,Littauthorid,score,FirstNameWD,LastNameWD,BirthDayWD,FirstNameSPA,LastNameSPA,BirthDaySPA,searchStr,BirthYearSPA,BirthYearWD,wd
0,Gottfrid Carlsson 1887,NaN,sj9PGLAlnmUAAAAAAA6R9w,CarlssonGottfrid,40.213776,Henrik,Carlsson,NaN,Gottfrid,Carlsson,NaN,NaN,1887.0,1887,Q5603107
1,Gottfrid Carlsson 1887,NaN,sj9PGLAlnmUAAAAAAA6R-A,CarlssonGottfrid,39.32846,Henrik,Carlsson,NaN,Gottfrid,Carlsson,NaN,NaN,1887.0,1887,Q5603107
2,Gottfrid Carlsson 1887,NaN,sj9PGLAlnmUAAAAAAA6Pqg,CarlssonGottfrid,25.88007,Henrik,Carlsson,NaN,E G,Carlsson,NaN,NaN,1887.0,1887,Q5603107
3,Gottfrid Carlsson 1887,NaN,sj9PGLAlnmUAAAAAAA6QqQ,CarlssonGottfrid,25.743935,Henrik,Carlsson,NaN,Erik,Carlsson,NaN,NaN,1887.0,1887,Q5603107
4,Gottfrid Carlsson 1887,NaN,sj9PGLAlnmUAAAAAAA6Q4g,CarlssonGottfrid,25.580647,Henrik,Carlsson,NaN,Erik,Carlsson,NaN,NaN,1887.0,1887,Q5603107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Sven Alfons 1918,NaN,rkIpfGqHXrAAAAAAAADoig,AlfonsS,21.41803,Sven,None,NaN,Sven,Lagerbäck,NaN,NaN,1918.0,1918,Q1814834
196,Sven Alfons 1918,NaN,I_2wtd5VefAAAAAAAAA_8w,AlfonsS,21.36111,Sven,None,NaN,Sven,Mattsson,NaN,NaN,1918.0,1918,Q1814834
197,Otto Vilhelm Klinckowström 1683,NaN,WQknIBFpuuAAAAAAAAN9ew,KlinckowstromO,67.93422,Otto,Klinckowström,NaN,,KlinckowströmOtto,NaN,NaN,1683.0,1683,Q2041231
198,Jan de Liefde 1814,NaN,seuNpYu6DKAAAAAAAABj0A,LiefdeJ,18.259153,Jan,None,NaN,Johan,Molin,NaN,NaN,1814.0,1814,Q2414590


In [42]:
dfCandidates = dfScoreSPALitt.sort_values(by=['score'], ascending=False)
dfCandidates

,all,WikidataID,spaid,Littauthorid,score,FirstNameWD,LastNameWD,BirthDayWD,FirstNameSPA,LastNameSPA,BirthDaySPA,searchStr,BirthYearSPA,BirthYearWD,wd
131,Joakim Vilhelm Liliestråle 1721,NaN,rkIpfGqHXrAAAAAAAAD18w,LiliestråleJW,82.0224,Joakim,Lilliestråle,NaN,Joakim,Liliestråle,NaN,NaN,1721.0,1721,Q5951400
1019,Anna Gustafsdotter Banér 1585,NaN,sj9PGLAlnmUAAAAAACYy8A,BanerA,80.52638,Anna,Banér,NaN,Anna,Gustafsdotter-Banér,NaN,NaN,1585,1585,Q22670927
20,Elis Chiewitz 1784,NaN,sj9PGLAlnmUAAAAAABVvkg,ChiewitzE,79.24202,Elis,Chiewitz,NaN,Elis,Chiewitz,NaN,NaN,1839.0,1784,Q5607767
139,Johan Fredrik Muncktell 1764,NaN,sj9PGLAlnmUAAAAAAB2V9Q,MuncktellJF,76.43912,Johan,Muncktell,NaN,Johan,Muncktell,NaN,NaN,1764.0,1764,Q5996948
460,Sten Arvidsson Sture 1681,NaN,mLh5P0pzYFAAAAAAAAA2kQ,ArvidssonSS,72.96724,Sten,Sture,NaN,Sten,Sture,NaN,NaN,1681,1681,Q52714485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,Mordekhai Forlerer 1895,NaN,sj9PGLAlnmUAAAAAAA7RTA,ForlererM,7.767416,None,None,NaN,Gunnar,Carlbäck,NaN,NaN,1895,1895,Q114101218
2502,Mordekhai Forlerer 1895,NaN,ciLMGSqrYjAAAAAAAABgyA,ForlererM,7.713413,None,None,NaN,,Witzansky,NaN,NaN,1895,1895,Q114101218
2503,Mordekhai Forlerer 1895,NaN,rkIpfGqHXrAAAAAAAADfKQ,ForlererM,7.7043,None,None,NaN,Oscar,Lindh,NaN,NaN,1895,1895,Q114101218
2504,Mordekhai Forlerer 1895,NaN,sj9PGLAlnmUAAAAAACeiXg,ForlererM,7.697796,None,None,NaN,Nils,Lindblad,NaN,NaN,1895,1895,Q114101218


<a href="#SPA"></a>
## Candidates SPA


In [43]:
#dfCandidates

In [44]:
from IPython.display import HTML

space = "%20"
dfCandidates["Littbank"] = "<a target=_blank href='https://litteraturbanken.se/f%C3%B6rfattare/" + dfCandidates["Littauthorid"] + "'>link Littbank</a>"

dfCandidates["WD"] = "<a target=_blank href='https://www.wikidata.org/wiki/" + dfCandidates["wd"] + "'>" + dfCandidates["wd"] + "</a>"
dfCandidates["WD_template"] = "{{Q|" + dfCandidates["wd"] + "}}"

dfCandidates["SPA"] = "<a target=_blank href='https://portrattarkiv.se/details/" + dfCandidates["spaid"] + "'>link SPA</a>"

dfCandidates["WikicommonsUrl"] = "<a target=_blank href='https://commons.wikimedia.org/wiki/Category:" + dfCandidates["FirstNameWD"] \
       + "_" + dfCandidates["LastNameWD"] + "_" + dfCandidates["BirthYearWD"] + "'>link Commons</a>"

dfCandidates["name_Ascii"] = dfCandidates["FirstNameWD"] +\
       space + dfCandidates["LastNameWD"] + space + dfCandidates["BirthYearWD"]
dfCandidates["name_str"] = dfCandidates["FirstNameWD"] \
       + " " + dfCandidates["LastNameWD"] + " " + dfCandidates["BirthYearWD"]
#dfCandidates["name_Ascii"] = dfCandidates["FirstNameWD"] \
#       + space + dfCandidates["LastNameWD"] + space + dfCandidates["BirthYearWD"]
dfCandidates["CommonsCat"] = "[[Category:" + dfCandidates["name_Ascii"] + "]]"

curlyLeftBrackets = "%7B%7B"
curlyRightBrackets = "%7D%7D"
verticalBar = "%7C"
lineFeed = "%0A"
equalSign = "%3D"
information = curlyLeftBrackets + "Information" 
description = verticalBar + "description" + equalSign + curlyLeftBrackets 
english = curlyLeftBrackets + "en" + verticalBar
swedish = curlyLeftBrackets + "sv" + verticalBar
uploadedFromSwe = "uppladdat%20fr%C3%A5n%20Portrattarkiv.se%20bild%20av%20"
uploadedFromEn = "uploaded%20from%20Portrattarkiv.se%20picture%20of%20"


source = verticalBar + "source" + equalSign + "https://portrattarkiv.se/details/"
author= verticalBar + "author" + equalSign + curlyLeftBrackets + "sv|okänd"+ curlyRightBrackets + \
     curlyLeftBrackets +"en|Unknown" + curlyRightBrackets + lineFeed
permission = verticalBar + "permission" + equalSign + lineFeed
otherversion = verticalBar + "other versions" + equalSign + lineFeed
categories = lineFeed + "[[Category:Uploaded with spa2CommonsNotebook]]" + lineFeed + \
             "[[Category:Swedish_Portrait_Archive]]"
license = "&wpLicense=cc-by-sa-4.0" 
wpDestFile = "&wpDestFile="
wpSourceType = "wpSourceType=url&wpUploadFileURL=https://portrattarkiv.se/endpoints/file.php?id="
    
dfCandidates["UploadUrl"] = "<a target=_blank href='" + \
      "https://commons.wikimedia.org/wiki/Special:Upload?wpUploadDescription" + \
       "=" + curlyLeftBrackets + "Information" + \
       lineFeed + verticalBar + "description" + equalSign + \
       swedish + uploadedFromSwe + dfCandidates["name_Ascii"] + space + dfCandidates["WD_template"] + curlyRightBrackets + lineFeed + \
       english + uploadedFromEn + dfCandidates["name_Ascii"] + space + dfCandidates["WD_template"] + curlyRightBrackets + lineFeed +  \
       source + dfCandidates["spaid"] + lineFeed + \
       author + \
       permission + \
       otherversion + curlyRightBrackets + lineFeed + lineFeed + \
       categories + lineFeed + \
       dfCandidates["CommonsCat"] + \
       license + \
       wpDestFile + dfCandidates["name_str"] + "_SPA_Notebook_1.jpg&" + \
       wpSourceType + dfCandidates["spaid"] + \
       "'>link Upload</a>"

pd.set_option("display.max.columns", None) 
HTML(dfCandidates[["score","name_str", "LastNameWD","FirstNameWD","Littbank", "UploadUrl","WD", "SPA", "spaid",\
                                   "WikicommonsUrl"]
                                 ].to_html(escape=False))

,score,name_str,LastNameWD,FirstNameWD,Littbank,UploadUrl,WD,SPA,spaid,WikicommonsUrl
131,82.0224,Joakim Lilliestråle 1721,Lilliestråle,Joakim,link Littbank,<a target=_blank href='https://commons.wikimedia.org/wiki/Special:Upload?wpUploadDescription=%7B%7BInformation%0A%7Cdescription%3D%7B%7Bsv%7Cuppladdat%20fr%C3%A5n%20Portrattarkiv.se%20bild%20av%20Joakim%20Lilliestråle%201721%20{{Q|Q5951400}}%7D%7D%0A%7B%7Ben%7Cuploaded%20from%20Portrattarkiv.se%20picture%20of%20Joakim%20Lilliestråle%201721%20{{Q|Q5951400}}%7D%7D%0A%7Csource%3Dhttps://portrattarkiv.se/details/rkIpfGqHXrAAAAAAAAD18w%0A%7Cauthor%3D%7B%7Bsv|okänd%7D%7D%7B%7Ben|Unknown%7D%7D%0A%7Cpermission%3D%0A%7Cother versions%3D%0A%7D%7D%0A%0A%0A[[Category:Uploaded with spa2CommonsNotebook]]%0A[[Category:Swedish_Portrait_Archive]]%0A[[Category:Joakim%20Lilliestråle%201721]]&wpLicense=cc-by-sa-4.0&wpDestFile=Joakim Lilliestråle 1721_SPA_Notebook_1.jpg&wpSourceType=url&wpUploadFileURL=https://portrattarkiv.se/endpoints/file.php?id=rkIpfGqHXrAAAAAAAAD18w'>link Upload,Q5951400,link SPA,rkIpfGqHXrAAAAAAAAD18w,link Commons
1019,80.52638,Anna Banér 1585,Banér,Anna,link Littbank,<a target=_blank href='https://commons.wikimedia.org/wiki/Special:Upload?wpUploadDescription=%7B%7BInformation%0A%7Cdescription%3D%7B%7Bsv%7Cuppladdat%20fr%C3%A5n%20Portrattarkiv.se%20bild%20av%20Anna%20Banér%201585%20{{Q|Q22670927}}%7D%7D%0A%7B%7Ben%7Cuploaded%20from%20Portrattarkiv.se%20picture%20of%20Anna%20Banér%201585%20{{Q|Q22670927}}%7D%7D%0A%7Csource%3Dhttps://portrattarkiv.se/details/sj9PGLAlnmUAAAAAACYy8A%0A%7Cauthor%3D%7B%7Bsv|okänd%7D%7D%7B%7Ben|Unknown%7D%7D%0A%7Cpermission%3D%0A%7Cother versions%3D%0A%7D%7D%0A%0A%0A[[Category:Uploaded with spa2CommonsNotebook]]%0A[[Category:Swedish_Portrait_Archive]]%0A[[Category:Anna%20Banér%201585]]&wpLicense=cc-by-sa-4.0&wpDestFile=Anna Banér 1585_SPA_Notebook_1.jpg&wpSourceType=url&wpUploadFileURL=https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAACYy8A'>link Upload,Q22670927,link SPA,sj9PGLAlnmUAAAAAACYy8A,link Commons
20,79.24202,Elis Chiewitz 1784,Chiewitz,Elis,link Littbank,<a target=_blank href='https://commons.wikimedia.org/wiki/Special:Upload?wpUploadDescription=%7B%7BInformation%0A%7Cdescription%3D%7B%7Bsv%7Cuppladdat%20fr%C3%A5n%20Portrattarkiv.se%20bild%20av%20Elis%20Chiewitz%201784%20{{Q|Q5607767}}%7D%7D%0A%7B%7Ben%7Cuploaded%20from%20Portrattarkiv.se%20picture%20of%20Elis%20Chiewitz%201784%20{{Q|Q5607767}}%7D%7D%0A%7Csource%3Dhttps://portrattarkiv.se/details/sj9PGLAlnmUAAAAAABVvkg%0A%7Cauthor%3D%7B%7Bsv|okänd%7D%7D%7B%7Ben|Unknown%7D%7D%0A%7Cpermission%3D%0A%7Cother versions%3D%0A%7D%7D%0A%0A%0A[[Category:Uploaded with spa2CommonsNotebook]]%0A[[Category:Swedish_Portrait_Archive]]%0A[[Category:Elis%20Chiewitz%201784]]&wpLicense=cc-by-sa-4.0&wpDestFile=Elis Chiewitz 1784_SPA_Notebook_1.jpg&wpSourceType=url&wpUploadFileURL=https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAABVvkg'>link Upload,Q5607767,link SPA,sj9PGLAlnmUAAAAAABVvkg,link Commons
139,76.43912,Johan Muncktell 1764,Muncktell,Johan,link Littbank,<a target=_blank href='https://commons.wikimedia.org/wiki/Special:Upload?wpUploadDescription=%7B%7BInformation%0A%7Cdescription%3D%7B%7Bsv%7Cuppladdat%20fr%C3%A5n%20Portrattarkiv.se%20bild%20av%20Johan%20Muncktell%201764%20{{Q|Q5996948}}%7D%7D%0A%7B%7Ben%7Cuploaded%20from%20Portrattarkiv.se%20picture%20of%20Johan%20Muncktell%201764%20{{Q|Q5996948}}%7D%7D%0A%7Csource%3Dhttps://portrattarkiv.se/details/sj9PGLAlnmUAAAAAAB2V9Q%0A%7Cauthor%3D%7B%7Bsv|okänd%7D%7D%7B%7Ben|Unknown%7D%7D%0A%7Cpermission%3D%0A%7Cother versions%3D%0A%7D%7D%0A%0A%0A[[Category:Uploaded with spa2CommonsNotebook]]%0A[[Category:Swedish_Portrait_Archive]]%0A[[Category:Johan%20Muncktell%201764]]&wpLicense=cc-by-sa-4.0&wpDestFile=Johan Muncktell 1764_SPA_Notebook_1.jpg&wpSourceType=url&wpUploadFileURL=https://portrattarkiv.se/endpoints/file.php?id=sj9PGLAlnmUAAAAAAB2V9Q'>link Upload,Q5996948,link SPA,sj9PGLAlnmUAAAAAAB2V9Q,link Commons
460,72.96724,Sten 

In [46]:
dfCandidates.to_csv("Littraturbanken_candidates.csv") 

### Test med preview